# Code to produce the cutouts for the MIRI data (DONE)

Let's start with the imports

In [ ]:
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS, FITSFixedWarning
from astropy.nddata import Cutout2D
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import warnings

warnings.simplefilter("ignore", category=FITSFixedWarning)



Let's look at the NIRCam documentation:

So we find that in the short filters we have 0.031"/pix and Amir used 167 x 167 pixels, which means he is using an image size of 5.177" for each cutout. We also know that the total FOV is around 63.74" x 63.74". Now we need to translate this to MIRI:

So taking this conversion of pixels to angular resolution we find that MIRI pixels are on average about 3.5 times larger than NIRCam pixels, so we just need to divide 167 by 3.55, which leaves us with ~48 pixels in MIRI.



# Define function to produce cutouts automatically

In [ ]:
def produce_cutouts(cat, indir, output_dir, survey, x_arcsec, filter, obs="", nan_thresh=0.4):
    """Function that reads in all the .fits files in a folder and produces cutouts, 
        which are then stored in the output directory

    Args:
        cat (string): catalogue.fits file that contains galaxy positions and IDs
        indir (string): Input directory for the .fits files from which the cutouts are taken
        output_dir (string): Output directory in which the cutouts should be stored
        survey (string): Either 'primer' for PRIMER or 'cweb' for COSMOS-Web
        x_arcsec (float): Side length of the cutouts in arcseconds.        
        filter (string): Specific filter used for the imaging
        obs(string): Observation number if necessary
        nan_thresh (float, optional): Relative amount of nan entries allowed in the data of 
            each cutout. Defaults to 0.4.
    """
    
    # Load target catalogue
    with fits.open(cat) as catalog_hdul:
        catalog_hdul.info()
        cat_data = catalog_hdul[1].data  # Extract data from table
        ids = cat_data['id']
        ra = cat_data['ra']
        dec = cat_data['dec']  
     
    filter_l = filter.lower()
    
    if '.fits' in indir:
        fits_files = [indir]
    else:
        fits_files = glob.glob(os.path.join(indir, f"*{filter_l}*.fits"))
        print(f"Found {len(fits_files)} FITS files from the {survey} survey with filter {filter} in directory {indir}.")
        print("Processing:")
    
    print("Found the following .fits files to take cutouts from:")
    for f in fits_files:
        print(f"{f}")
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    print(f"Files will be saved to {output_dir}.")

    counts = 0
    total = len(ra)
    
    if "rot" in indir: index = 0
    elif 'aligned' in indir: index = 0
    else: index = 1
    
    for data in fits_files:
        with fits.open(data) as miri_hdul:
            miri_hdul.info()
            miri_data = miri_hdul[index].data
            miri_header = miri_hdul[index].header
            wcs = WCS(miri_header)
            
            # Estimate pixel scale (arcsec/pixel)
            try:
                # CD matrix case
                cd1 = miri_header.get('CD1_1')
                cd2 = miri_header.get('CD2_2')
                if cd1 is not None and cd2 is not None:
                    pixel_scale = np.abs(cd1 + cd2) / 2 * 3600  # deg/pix -> arcsec/pix
                else:
                    # Fallback to CDELT
                    cdelt1 = miri_header.get('CDELT1')
                    cdelt2 = miri_header.get('CDELT2')
                    if cdelt1 is not None and cdelt2 is not None:
                        pixel_scale = np.abs(cdelt1 + cdelt2) / 2 * 3600
                    else:
                        raise ValueError("Cannot determine pixel scale from header.")
            except Exception as e:
                print(f"Error determining pixel scale: {e}")
                continue

            # Now compute pixel size for the desired arcsec cutout
            x_pixels = int(np.round(x_arcsec / pixel_scale))
            cutout_size = (x_pixels, x_pixels)

            # Loop over all galaxies first
            for i in range(len(ra)):
                target_coord = SkyCoord(ra[i], dec[i], unit=(u.deg, u.deg))
                found = False  # Track if the galaxy was mapped

                # Check if the galaxy is inside the mapped region
                x, y = wcs.world_to_pixel(target_coord)
                if (0 <= x < miri_data.shape[1]) and (0 <= y < miri_data.shape[0]):
                    found = True  # The galaxy is in the mapped region

                    # Extract the cutout
                    cutout = Cutout2D(miri_data, target_coord, cutout_size, wcs=wcs, mode="partial")

                    # Ensure cutout is valid
                    if cutout.data is None or cutout.data.size == 0:
                        #print(f"Skipping {ids[i]}: Empty cutout")
                        continue
                    
                    # Check for NaN ratio
                    nans = np.isnan(cutout.data).sum()
                    nan_ratio = nans / cutout.data.size
                    
                    if nan_ratio < nan_thresh:
                        # Save PNG preview
                        plt.figure(figsize=(6, 6))
                        plt.imshow(cutout.data, origin="lower", cmap="gray")
                        plt.colorbar()
                        plt.title(f'{survey}: {ids[i]} at {filter}')

                        
                        png_filename = os.path.join(output_dir, f"{ids[i]}_{filter}_cutout_{survey}{obs}.png")
                        plt.savefig(png_filename)
                        plt.close()

                        counts += 1

                        # Adjust header CRPIX values for the cutout
                        # Get the new reference pixel
                        new_crpix = cutout.wcs.world_to_pixel(target_coord)
                        new_header = cutout.wcs.to_header()
                        new_header['CRPIX1'] = int(np.round(new_crpix[0]))
                        new_header['CRPIX2'] = int(np.round(new_crpix[1]))
                        
                        # Save the cutout FITS file
                        hdu_list = fits.HDUList()
                        hdu_list.append(fits.PrimaryHDU(header=miri_hdul[0].header))
                        hdu_list.append(fits.ImageHDU(data=cutout.data, header=cutout.wcs.to_header()))

                        for ext in range(2, len(miri_hdul)):
                            if miri_hdul[ext].data is not None:
                                hdu_list.append(fits.ImageHDU(data=miri_hdul[ext].data, header=miri_hdul[ext].header))

                        fits_filename = os.path.join(output_dir, f"{ids[i]}_{filter}_cutout_{survey}{obs}.fits")
                        hdu_list.writeto(fits_filename, overwrite=True)

                            #print(f"Saved: {ids[i]} at {filter}")
                    else:
                        #print(f"Skipping {ids[i]}: Too many NaNs ({nan_ratio:.2f})")
                        continue
                else:
                    continue  # Galaxy is outside the mapped region

            if not found:
                print(f"Skipping {ids[i]}: Not found in any FITS file")
                continue
        
    print(f"Produced cutouts for {counts} of {total} galaxies in the catalogue.")

        

# Old function to produce cutouts

In [ ]:
def produce_cutouts_old(cat, indir, output_dir, survey, x_pixels, filter, obs="", nan_thresh=0.4, suffix=''):
    """Function that reads in all the .fits files in a folder and produces cutouts, 
        which are then stored in the output directory

    Args:
        cat (string): catalogue.fits file that contains galaxy positions and IDs
        indir (string): Input directory for the .fits files from which the cutouts are taken
        output_dir (string): Output directory in which the cutouts should be stored
        survey (string): Either 'primer' for PRIMER or 'cweb' for COSMOS-Web
        x_pixels (int): Side length of the cutouts in pixels (produces a square frame)
        filter (string): Specific filter used for the imaging
        nan_thresh (float, optional): Relative amount of nan entries allowed in the data of 
            each cutout. Defaults to 0.4.
    """
    
    # Load target catalogue
    with fits.open(cat) as catalog_hdul:
        catalog_hdul.info()
        cat_data = catalog_hdul[1].data  # Extract data from table
        ids = cat_data['id']
        ra = cat_data['ra']
        dec = cat_data['dec']  
    
    filter_l = filter.lower()
    
    fits_files = glob.glob(os.path.join(indir, f"*{filter_l}*.fits"))
    print(f"Found {len(fits_files)} FITS files from the {survey} survey with filter {filter}.")
    print("Processing:")
    for f in fits_files:
        print(f"{f}")

    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    print(f"Files will be saved to {output_dir}.")

    counts = 0
    total = len(ra)

    cutout_size = (x_pixels, x_pixels)
    
    for data in fits_files:
        with fits.open(data) as miri_hdul:
            miri_data = miri_hdul[1].data
            miri_header = miri_hdul[1].header
            wcs = WCS(miri_header)

        # Loop over all galaxies first
        for i in range(len(ra)):
            target_coord = SkyCoord(ra[i], dec[i], unit=(u.deg, u.deg))
            found = False  # Track if the galaxy was mapped

            # Check if the galaxy is inside the mapped region
            x, y = wcs.world_to_pixel(target_coord)
            if (0 <= x < miri_data.shape[1]) and (0 <= y < miri_data.shape[0]):
                found = True  # The galaxy was mapped

                # Extract the cutout
                cutout = Cutout2D(miri_data, target_coord, cutout_size, wcs=wcs, mode="partial")

                # Ensure cutout is valid
                if cutout.data is None or cutout.data.size == 0:
                    #print(f"Skipping {ids[i]}: Empty cutout")
                    continue
                
                # Check for NaN ratio
                nans = np.isnan(cutout.data).sum()
                nan_ratio = nans / cutout.data.size
                
                
                if nan_ratio < nan_thresh:
                    # Save PNG preview
                    plt.figure(figsize=(6, 6))
                    plt.imshow(cutout.data, origin="lower", cmap="gray")
                    plt.colorbar()
                    plt.title(f'{survey}: {ids[i]} at {filter}')

                    png_filename = os.path.join(output_dir, f"{ids[i]}_{filter}_cutout_{survey}{obs}_{suffix}.png")
                    plt.savefig(png_filename)
                    plt.close()

                    counts += 1

                    # Adjust header CRPIX values for the cutout
                    # Get the new reference pixel
                    new_crpix = cutout.wcs.world_to_pixel(target_coord)
                    new_header = cutout.wcs.to_header()
                    new_header['CRPIX1'] = int(np.round(new_crpix[0]))
                    new_header['CRPIX2'] = int(np.round(new_crpix[1]))
                    
                    # Save the cutout FITS file
                    hdu_list = fits.HDUList()
                    hdu_list.append(fits.PrimaryHDU(header=miri_hdul[0].header))
                    hdu_list.append(fits.ImageHDU(data=cutout.data, header=cutout.wcs.to_header()))

                    for ext in range(2, len(miri_hdul)):
                        if miri_hdul[ext].data is not None:
                            hdu_list.append(fits.ImageHDU(data=miri_hdul[ext].data, header=miri_hdul[ext].header))

                    fits_filename = os.path.join(output_dir, f"{ids[i]}_{filter}_cutout_{survey}{obs}_{suffix}.fits")
                    hdu_list.writeto(fits_filename, overwrite=True)

                    #print(f"Saved: {ids[i]} at {filter}")
                else:
                    #print(f"Skipping {ids[i]}: Too many NaNs ({nan_ratio:.2f})")
                    continue
            else:
                continue  # Galaxy is outside the mapped region

        if not found:
            #print(f"Skipping {ids[i]}: Not found in any FITS file")
            continue
        
    print(f"Produced cutouts for {counts} of {total} galaxies in the catalogue.")

# Create cutouts of the unmodified FITS files

Original data:

In [ ]:
# Define directories
primer003 =  '/home/bpc/University/master/Red_Cardinal/MIRI/PRIMER_003/'
primer004 =  '/home/bpc/University/master/Red_Cardinal/MIRI/PRIMER_004/'
cweb1 =   '/home/bpc/University/master/Red_Cardinal/MIRI/COSMOS-Web_1/'
cweb2 =   '/home/bpc/University/master/Red_Cardinal/MIRI/COSMOS-Web_2/'

catalogue =  '/home/bpc/University/master/Red_Cardinal/cat_targets.fits'

output_dir = '/home/bpc/University/master/Red_Cardinal/cutouts/'

# Produce cutouts for all surveys
produce_cutouts_old(catalogue, primer003, output_dir, 'primer', 48, 'F770W', '003')
produce_cutouts_old(catalogue, primer003, output_dir, 'primer', 48, 'F1800W', '003')

produce_cutouts_old(catalogue, primer004, output_dir, 'primer', 48, 'F770W', '004')
produce_cutouts_old(catalogue, primer004, output_dir, 'primer', 48, 'F1800W', '004')

produce_cutouts_old(catalogue, cweb1, output_dir, 'cweb', 48, 'F770W', '1')
produce_cutouts_old(catalogue, cweb2, output_dir, 'cweb', 48, 'F770W', '2')




Shifted data:

In [ ]:
# Define directories
primer003 =  '/home/bpc/University/master/Red_Cardinal/MIRI_shifted/PRIMER_003_shifted/'
primer004 =  '/home/bpc/University/master/Red_Cardinal/MIRI_shifted/PRIMER_004_shifted/'
cweb1 =   '/home/bpc/University/master/Red_Cardinal/MIRI_shifted/COSMOS-Web_1_shifted/'
cweb2 =   '/home/bpc/University/master/Red_Cardinal/MIRI_shifted/COSMOS-Web_2_shifted/'

catalogue =  '/home/bpc/University/master/Red_Cardinal/cat_targets.fits'

output_dir = '/home/bpc/University/master/Red_Cardinal/cutouts_shifted/'

# Produce cutouts for all surveys
produce_cutouts_old(catalogue, primer003, output_dir, 'primer', 48, 'F770W', '003', suffix='shifted')
produce_cutouts_old(catalogue, primer003, output_dir, 'primer', 48, 'F1800W', '003', suffix='shifted')

produce_cutouts_old(catalogue, primer004, output_dir, 'primer', 48, 'F770W', '004', suffix='shifted')
produce_cutouts_old(catalogue, primer004, output_dir, 'primer', 48, 'F1800W', '004', suffix='shifted')

produce_cutouts_old(catalogue, cweb1, output_dir, 'cweb', 48, 'F770W', '1', suffix='shifted')
produce_cutouts_old(catalogue, cweb2, output_dir, 'cweb', 48, 'F770W', '2', suffix='shifted')




# Create cutouts!!

Note that the number of MIRI pixels needs to be reduced from 48 to 45 when dealing with the expanded and rotated .fits file as a source of the cutouts. This is due to the resampling of the pixels.

In [ ]:
# Define directories
primer003 =  '/home/bpc/University/master/Red_Cardinal/MIRI_rotated/PRIMER_003_rot/'
primer004 =  '/home/bpc/University/master/Red_Cardinal/MIRI_rotated/PRIMER_004_rot/'
cweb1 =   '/home/bpc/University/master/Red_Cardinal/MIRI_rotated/COSMOS-Web_1_rot/'
cweb2 =   '/home/bpc/University/master/Red_Cardinal/MIRI_rotated/COSMOS-Web_2_rot/'

catalogue =  '/home/bpc/University/master/Red_Cardinal/cat_targets.fits'

output_dir = '/home/bpc/University/master/Red_Cardinal/cutouts_rotated/'

# Produce cutouts for all surveys
#produce_cutouts(catalogue, primer003, output_dir, 'primer', 5.177, 'F770W', '003')
#produce_cutouts(catalogue, primer003, output_dir, 'primer', 5.177, 'F1800W', '003')

#produce_cutouts(catalogue, primer004, output_dir, 'primer', 5.177, 'F770W', '004')
#produce_cutouts(catalogue, primer004, output_dir, 'primer', 5.177, 'F1800W', '004')

#produce_cutouts(catalogue, cweb1, output_dir, 'cweb', 5.177, 'F770W', '1')
#produce_cutouts(catalogue, cweb2, output_dir, 'cweb', 5.177, 'F770W', '2')




Let's try and produce cutouts from the (hopefully) aligned FITS files COSMOS-Web 2:

In [ ]:
# Define directories
primer003_aligned =  '/home/bpc/University/master/Red_Cardinal/MIRI_aligned/PRIMER_003_aligned/'
primer004_aligned =  '/home/bpc/University/master/Red_Cardinal/MIRI_aligned/PRIMER_004_aligned/'
cweb1_aligned =   '/home/bpc/University/master/Red_Cardinal/MIRI_aligned/COSMOS-Web_1_aligned/'
cweb2_aligned =   '/home/bpc/University/master/Red_Cardinal/MIRI_aligned/COSMOS-Web_2_aligned/'

catalogue =  '/home/bpc/University/master/Red_Cardinal/cat_targets.fits'

output_dir = '/home/bpc/University/master/Red_Cardinal/cutouts_aligned/'

produce_cutouts(catalogue, primer003_aligned, output_dir, 'primer', 45, 'F770W', '003')
produce_cutouts(catalogue, primer003_aligned, output_dir, 'primer', 45, 'F1800W', '003')

produce_cutouts(catalogue, primer004_aligned, output_dir, 'primer', 45, 'F770W', '004')
produce_cutouts(catalogue, primer004_aligned, output_dir, 'primer', 45, 'F1800W', '004')

produce_cutouts(catalogue, cweb1_aligned, output_dir, 'cweb', 45, 'F770W', '1')
produce_cutouts(catalogue, cweb2_aligned, output_dir, 'cweb', 45, 'F770W', '2')


Apparently there's a problem when creating the following aligned cutouts:

Files in ./cutouts_rotated/ but not in ./cutouts_aligned/:
11723_F1800W_cutout_primer004_rot.fits
11723_F1800W_cutout_primer004_rot.png
11853_F770W_cutout_primer004_rot.fits
11853_F770W_cutout_primer004_rot.png
12717_F1800W_cutout_primer004_rot.fits
12717_F1800W_cutout_primer004_rot.png
13174_F1800W_cutout_primer004_rot.fits
13174_F1800W_cutout_primer004_rot.png

After visual inspection these cutouts do not seem to satisfy the nan-threshold anymore after the offset is corrected. So all good!

Now we produce the 3x3" cutouts:

In [ ]:
# Define directories
primer003 =  '/home/bpc/University/master/Red_Cardinal/MIRI_aligned/PRIMER_003_aligned/'
primer004 =  '/home/bpc/University/master/Red_Cardinal/MIRI_aligned/PRIMER_004_aligned/'
cweb1 =   '/home/bpc/University/master/Red_Cardinal/MIRI_aligned/COSMOS-Web_1_aligned/'
cweb2 =   '/home/bpc/University/master/Red_Cardinal/MIRI_aligned/COSMOS-Web_2_aligned/'

catalogue =  '/home/bpc/University/master/Red_Cardinal/cat_targets.fits'

output_dir = '/home/bpc/University/master/Red_Cardinal/cutouts_3x3/'

# Produce cutouts for all surveys
produce_cutouts(catalogue, primer003, output_dir, 'primer', 3, 'F770W')
produce_cutouts(catalogue, primer003, output_dir, 'primer', 3, 'F1800W')

produce_cutouts(catalogue, primer004, output_dir, 'primer', 3, 'F770W')
produce_cutouts(catalogue, primer004, output_dir, 'primer', 3, 'F1800W')

produce_cutouts(catalogue, cweb1, output_dir, 'cweb', 3, 'F770W')
produce_cutouts(catalogue, cweb2, output_dir, 'cweb', 3, 'F770W')


